In [1]:
from gsmodutils import GSMProject
import cameo
import cobra

In [2]:
project = GSMProject('.')  # load project from current directory

In [3]:
m = project.models[1]  # load E. coli GSM iJO1366 (https://www.embopress.org/doi/full/10.1038/msb.2011.65)

In [4]:
m.exchanges

[<Reaction EX_cm_e at 0x7f8b455400b8>,
 <Reaction EX_cmp_e at 0x7f8b45540198>,
 <Reaction EX_co2_e at 0x7f8b45540278>,
 <Reaction EX_cobalt2_e at 0x7f8b45540358>,
 <Reaction EX_colipa_e at 0x7f8b45540898>,
 <Reaction EX_glc__D_e at 0x7f8b45540a58>,
 <Reaction EX_glcn_e at 0x7f8b45540b38>,
 <Reaction EX_glcr_e at 0x7f8b45506320>,
 <Reaction EX_colipap_e at 0x7f8b45506400>,
 <Reaction EX_glcur_e at 0x7f8b455064e0>,
 <Reaction EX_glcur1p_e at 0x7f8b455065c0>,
 <Reaction EX_12ppd__R_e at 0x7f8b454ac6d8>,
 <Reaction EX_gln__L_e at 0x7f8b454ac7b8>,
 <Reaction EX_cpgn_e at 0x7f8b454ac898>,
 <Reaction EX_glu__L_e at 0x7f8b454ac978>,
 <Reaction EX_gly_e at 0x7f8b454aca58>,
 <Reaction EX_glyald_e at 0x7f8b454acb38>,
 <Reaction EX_glyb_e at 0x7f8b454acc18>,
 <Reaction EX_glyc_e at 0x7f8b454accf8>,
 <Reaction EX_12ppd__S_e at 0x7f8b454acdd8>,
 <Reaction EX_14glucan_e at 0x7f8b454aceb8>,
 <Reaction EX_cpgn_un_e at 0x7f8b454acf98>,
 <Reaction EX_15dap_e at 0x7f8b454b70b8>,
 <Reaction EX_glyc__R_e at